<center>
<!-- ![sesion](images/session_example.png) -->
<img src="images/session_example.png" width="600">
<h1>Recomendacion Basada en Sesiones</h1>
<h3>Sebastian Prillo</h3>
<h4>Miercoles 19 de septiembre de 2018 - Seminario Machin Lenin</h4>
</center>

# Por que Reco?

***[...] the recommendations were used so extensively by Amazon.com that a Microsoft Research report estimated 30 percent of Amazon.com's page views were from recommendations. Similarly, Netflix used recommender systems so extensively that their Chief Product Officer, Neil Hunt, indicated that more than 80 percent of movies watched on Netflix came through recommendations, and placed the value of Netflix recommendations at more than US \$1 billion per year.*** ([Amazon 2017](https://www.computer.org/csdl/mags/ic/2017/03/mic2017030012.pdf))


# Problematica tipica
- Un usario navega productos en un sitio de e-commerce. Queremos recomendarle productos que le puedan interesar.
<center>
<img src="images/session_example.png" width="600">
</center>
- Metrica de negocio concreta: maximizar el **click-through-rate** (a.k.a. CTR) de nuestro recomendador:
$$CTR = \frac{\text{# recomendaciones clickeadas}}{\text{# recomendaciones vistas}}$$
(el objetivo podria ser distinto, como maximizar la cantidad de ventas, esto es solo a modo de motivacion para tener una metrica de negocio en la cabeza)

**Pregunta**: Hay alguna forma de estimar el CTR de un modelo sin ponerlo en produccion? (a.k.a. offline)

# Particularidades del problema

No contamos con informacion del usuario, porque:

- El usuario no esta loggeado/identificado/autenticado
- La informacion historica del usuario no es relevante
- etc

$\Rightarrow$ La unidad de trabajo es la **sesion**.

# Baselines: User-knn?

- Para generar recomendaciones para un usuario, busco **usuarios similares** (que tengan muchos productos en comun) y recomiendo los demas productos de esos usuarios.
- Los primeros sistemas de recomendacion fueron de este tipo

**No podemos usarlo en nuestro problema por motivos obvios**

# Baselines: Item-knn

- Paper: [Amazon 2003](https://www.cs.umd.edu/~samir/498/Amazon-Recommendations.pdf)
- Para cada item, busco **items similares** (que se hayan consumido en simultaneo mas de lo esperado al azar)
- Para generar recomendaciones para un usuario, busco items similares a los que consumio.
- Escala mucho mejor que user-based CF
- Adoptado de forma masiva en la industria

Podemos adaptarlo a nuestro problema.

# Baselines: Item-knn

Defino la similitud entre dos items como:

$$sim(i, j) = \frac{c_{i,j}}{\sqrt{c_i} \sqrt{c_j}}$$

donde

$$c_i = \text{cantidad de sesiones donde $i$ aparece}$$

$$c_{i,j} = \text{cantidad de sesiones donde $i$ y $j$ aparecen}$$

**Nota**: Hay mas formas posibles de definir la similitud.

**Nota**: Hay que tener cuidado con las estimaciones ruidosas.

Genero los candidatos mirando los items mas similares a los que aparecieron en la sesion (o al ultimo de la sesion nada mas).

# Baselines: Item-knn

***This baseline is one of the most common item-to-item solutions in practical systems, that provides recommendations in the “others who viewed this item also viewed these ones” setting. Despite of its simplicity it is usually a strong baseline (Linden et al., 2003; Davidson et al., 2010).*** ([Hidasi 2016](https://arxiv.org/pdf/1511.06939.pdf))

***The algorithm scales to hundreds of millions of users and tens of millions of items without sampling or other techniques that can reduce the quality of the recommendations.*** ([Amazon 2017](https://www.computer.org/csdl/mags/ic/2017/03/mic2017030012.pdf))

# Propuesta para resolver el problema

Resolvemos en cambio el problema de  **prediccion del proximo producto de la secuencia (dados los anteriores)**.

Una vez entrenado el modelo, usamos las probabilidades del modelo sobre el proximo producto para generar candidatos de recomendacion.

# Propuesta para resolver el problema: caveat

**Observacion**: Este problema **no es equivalente al original** (maximizar CTR):
- Cruzamos los dedos para que resolver bien el nuevo problema implique resolver bien el original.
- La ventaja es que la metrica del problema nuevo (precision) la podemos calcular **facil a partir de datos historicos** (offline), mientras que el CTR no!

***It is important to emphasize that recommendation often involves solving a surrogate problem and transferring the result to a particular context. A classic example is the assumption that accurately predicting ratings leads to effective movie recommendations. We have found that the choice of this surrogate learning problem has an outsized importance on performance in A/B testing but is very difficult to measure with offline experiments*** ([YouTube 2016](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45530.pdf))

# Formalizando

Sea:

- $I$: el conjunto de items/productos ($|I| = n$)
- $S^{(i)}$: la $i$-esima sesion de entre $m$, con $S^{(i)} = (S_1^{(i)}, \dots, S_{l_i}^{(i)}) \in I^{l_i}$.

Buscamos una funcion (modelo)

$$f : \cup_k I^k \rightarrow Prob(I)$$

que dada la secuencia de productos vistos por el usuario, nos de una distribucion de probabilidad sobre el siguiente producto. 

**Objetivo**: maximizar la precision (o precision at $K$, etc.)

# Cadenas de Markov

- Considerar solo el ultimo producto.
- Se calcula la matriz de probabilidad de transicion entre productos.

Ej: Dadas sesiones:
- $\text{ipod} \rightarrow \text{auriculares}$
- $\text{celular} \rightarrow \text{auriculares} \rightarrow \text{memoria 16GB}$
- $\text{celular} \rightarrow \text{memoria 16GB} \rightarrow \text{auriculares}$

El modelo es:

|actual\siguiente|ipod|celular|auriculares|memoria 16GB|
|-|-|-|-|-|
|ipod|0.0|0.0|1.0|0.0|
|celular|0.0|0.0|0.5|0.5|
|auriculares|0.0|0.0|0.0|1.0|
|memoria 16GB|0.0|0.0|1.0|0.0|

# Cadenas de Markov: Problemas

- Las probabilidades de transicion son solo **estimaciones de la realidad** y son pesimas para los productos que aparecen pocas veces.
- Otra forma de verlo: Sea $t$ la cantidad promedio de transiciones relevantes desde un producto a otro. Entonces el modelo de Markov tiene que aprender $tN$ parametros. Es mucho! Y la mayoria de ellos se infiere a partir de poca data $\Rightarrow$ overfitting.

<center>
<img src="images/long_tail.jpg" width="400">
</center>

# Cadenas de Markov Factorizadas

Paper: [Rendle 2010](https://www.ismll.uni-hildesheim.de/pub/pdfs/RendleFreudenthaler2010-FPMC.pdf)

Motivacion: Supongamos que tenemos tres sesiones:
- $\text{celular} \rightarrow \text{auricular 1} \rightarrow \text{memoria 16 GB}$
- $\text{celular} \rightarrow \text{auricular 1} \rightarrow \text{memoria 32 GB}$
- $\text{celular} \rightarrow \text{auricular 2} \rightarrow \text{memoria 16 GB}$

Estaria bueno que el modelo se de cuenta solo de que auricular 1 y auricular 2 son **'lo mismo'**. Si lo logramos, todas las transiciones desde/hacia auricular 1 van a contribuir a las estimaciones de las probabilidades de transicion de auricular 2 y viceversa.

- Asi aprenderiamos que de auricular 2 podemos transicionar a memoria 32 GB.
- Es como si aumentaramos nuestro dataset $\Rightarrow$ reduzco overfitting, generalizo mejor.

# Digresion: Embeddings

Dado un conjunto $X$ (ej: de productos), un **embedding** es una funcion $emb : X \rightarrow \mathbb R^k$.
- La idea es que con un embedding bueno, elementos 'semanticamente similares' van a parar a vectores similares.
- $k$ se llama la **dimension** del embedding.
- Hay que definir una nocion de 'similitud' en $\mathbb R^k$, tipicamente el producto interno. ($sim(u, v) = \left<u, v\right>$)

Ejemplo: embeddings de dimension 2 para palabras:

<center>
<img src="images/embeddings.png" width="500">
</center>

# Cadenas de Markov Factorizadas: Embeddings

- Para cada producto $i$ aprenderemos un embedding $v_i \in \mathbb R^k$. La idea va a ser que la probabilidad de transicionar de $i$ a $j$ sea proporcional a $sim(v_i, v_j)$ 
- En el ejemplo anterior, buscamos obtener algo como:

<center>
<img src="images/embedding_productos.png" width="400">
</center>

- Asi, por mas que no observemos una transicion de auricular 2 a memoria 32 GB, si auricular 1 y auricular 2 van a parar cerca, deducimos la transicion

# Cadenas de Markov Factorizadas: Formalizacion

Dados los embeddings $v_i$, la probabilidad de transicion del producto $i$ al $j$ en el modelo es

$$P(j | i) \propto
e^{\left<v_i, v_j\right>}$$

O sea,

$$P(j | i) = \frac{e^{\left<v_i, v_j\right>}}{\sum_{k} e^{\left<v_i, v_k\right>}}$$

tambien conocido como un **softmax**.

Ahora maximizar la probabilidad de los datos (bajo el modelo generativo asociado):

\begin{align}
\arg \max_{v_1,\dots,v_n} P(S^{(1)}, \dots, S^{(m)}) = \arg \max_{v_1,\dots,v_n}  \prod_{i = 1}^m \prod_{j = 1}^{l_i - 1} P({S_j^{(i)}, S_{j+1}^{(i)}})
\end{align}

O sea, hacer maximum likelihood.

***This factorization approach results
in less parameters and due to generalization to a
better quality than full parametrized models.*** ([Rendle 2010](https://www.ismll.uni-hildesheim.de/pub/pdfs/RendleFreudenthaler2010-FPMC.pdf))

# Cadenas de Markov Factorizadas: Notas

- Es muy parecido al skip-gram model para word embeddings. ([Mikolov 2013](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf))

- Se puede tener dos embeddings $u_i, v_i$ para cada producto, y definir en cambio:

$$P(j | i) \propto
e^{\left<u_i, v_j\right>}$$

- Entrenar con ranking loss suele andar mejor (se correlaciona mejor con la metrica de interes, como AUC) ([Rendle 2009](https://arxiv.org/pdf/1205.2618.pdf))

# Cadenas de Markov Factorizadas: Resultados

<center>
<img src="images/FMC2.png" width="500">
</center>

([Rendle 2010](https://www.ismll.uni-hildesheim.de/pub/pdfs/RendleFreudenthaler2010-FPMC.pdf))

# Recap

- Cadenas de Markov: Estimo $P(j|i)$ sin restricciones.
- Cadenas de Markov Factorizadas: Restrinjo $P$ tal que $P(j | i) \propto
e^{\left<v_i, v_j\right>}$ $\Rightarrow$ + bias, - variance $\Rightarrow$ - overfitting

# Redes Recurrentes
- Generalizan las Cadenas de Markov Factorizadas.

<center>
<img src="images/RNN.png" width="500">
</center>

- $i_t$ = input en tiempo $t$.
- $h_t$ = estado en tiempo $t$ $= f(h_{t - 1}, i_t)$ para alguna $f$. Ejemplos tipicos de $f$: GRU, LSTM.
- $o_t$ = output en tiempo $t$ $= g(h_t)$ para alguna $g$. Ejemplos tipicos de $g$: softmax.

# Cadenas de Markov Factorizadas como Redes Recurrentes

<center>
<img src="images/FMC_as_RNN.png" width="500">
</center>

- $i_t = v_t$
- $h_t = f(h_{t - 1}, i_t) = i_t$
- $o_t = g(h_t) = softmax(\left<h_t, v_1\right>, \dots, \left<h_t, v_m\right>) = softmax(V h_t)$

# Redes Recurrentes: GRU

<center>
<img src="images/RNN.png" width="500">
</center>

- $h_0 = 0$
- $i_t = v_t$
- $h_t = f(h_{t - 1}, i_t) = (1 - z_t) \odot h_{t-1} + z_t \odot \sigma_h(W_h x_t + U_h(r_t \odot h_{t-1}) + b_h)$ donde
$$z_t = \sigma_g(W_z x_t + U_z h_{t-1} + b_z)$$
$$r_t = \sigma_g(W_r x_t + U_r h_{t-1} + b_r)$$
$$\sigma_g(x) = \frac{1}{1 + e^{-x}}$$
$$\sigma_h(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$$
- $o_t = g(h_t) = softmax(\left<h_t, v_1\right>, \dots, \left<h_t, v_m\right>) = softmax(V h_t)$

# Redes Recurrentes: Algo Intermedio

<center>
<img src="images/RNN.png" width="500">
</center>

Promedio ponderado de la historia:

- $h_0 = v_1$
- $i_t = v_t$
- $h_t = \frac{i_t + h_{t-1}}{2}$
- $o_t = g(h_t) = softmax(\left<h_t, v_1\right>, \dots, \left<h_t, v_m\right>) = softmax(V h_t)$

La diferencia entre esto y la Cadena de Markov Factorizada es que aca usamos

$$\frac{v_1}{2^{t-1}} + \frac{v_2}{2^{t-1}} + \frac{v_3}{2^{t-2}} + \dots + \frac{v_t}{2}$$

para el estado en vez de $v_t$

# Redes Recurrentes: Resultados - Baselines

<center>
<img src="images/baselines.png" width="500">
</center>

([Hidasi 2016](https://arxiv.org/pdf/1511.06939.pdf))

- **POP** es recomendar lo mas popular.
- **S-POP** es recomendar lo mas popular de la sesion.
- **Item-KNN** es el item-knn discutido, mirando el ultimo producto nada mas para hacer la recomendacion.
- **BPR-MF** es como la red recurrente anterior pero tomando promedio no ponderado, y entrenando con ranking loss.

# Redes Recurrentes: Resultados

<center>
<img src="images/resultados_rnn.png" width="600">
</center>

([Hidasi 2016](https://arxiv.org/pdf/1511.06939.pdf))

# Redes (No) Recurrentes: YouTube 2016

<center>
<img src="images/reco_funnel.png" width="400">
</center>

([YouTube 2016](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45530.pdf))

# Redes (No) Recurrentes: YouTube 2016 - Candidate Generation Network

<center>
<img src="images/youtube_2016_candidate_generation.png" width="500">
</center>

([YouTube 2016](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45530.pdf))

# Redes (No) Recurrentes: YouTube 2016 - Candidate Generation Network - Resultados

<center>
<img src="images/generator_network_results.png" width="500">
</center>

([YouTube 2016](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45530.pdf))

# Redes (No) Recurrentes: YouTube 2016 - Ranking Network

<center>
<img src="images/youtube_2016_ranking.png" width="500">
</center>

([YouTube 2016](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45530.pdf))

# Redes (No) Recurrentes: YouTube 2016 - Ranking Network - Resultados

<center>
<img src="images/ranking_network_results.png" width="500">
</center>

([YouTube 2016](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45530.pdf))

# Referencias

- [Two Decades of Recommender Systems at Amazon.com (Amazon 2017)](https://www.computer.org/csdl/mags/ic/2017/03/mic2017030012.pdf)
- [Item-to-Item Collaborative Filtering (Amazon 2003)](https://www.cs.umd.edu/~samir/498/Amazon-Recommendations.pdf)
- [Session-Based Recommendations with Recurrent Neural Networks (Hidasi 2016)](https://arxiv.org/pdf/1511.06939.pdf)
- [Deep Neural Networks for YouTube Recommendations (YouTube 2016)](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45530.pdf)
- [Factorizing Personalized Markov Chains for Next-Basket Recommendation (Rendle 2010)](https://www.ismll.uni-hildesheim.de/pub/pdfs/RendleFreudenthaler2010-FPMC.pdf)
- [Distributed Representations of Words and Phrases
and their Compositionality (Mikolov 2013)](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)
- [BPR: Bayesian Personalized Ranking from Implicit Feedback (Rendle 2009)](https://arxiv.org/pdf/1205.2618.pdf)
